In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from wrappers import *
from typing import List

pd.options.mode.copy_on_write = True

In [2]:
from IPython.display import display, Markdown


def display_table_as_markdown_with_headers(df: pd.DataFrame, headers: List[str]):
    dataframe = df.copy()
    dataframe.columns = headers
    markdown_table = dataframe.to_markdown()
    display(Markdown(markdown_table))

In [3]:
TRIANGLE_MEASUREMENTS_PATH = Path("triangle-measurements-data.csv")
assert TRIANGLE_MEASUREMENTS_PATH.exists()

# Surowe wyniki pomiarów

In [4]:
raw_triangles_measurements = pd.read_csv(TRIANGLE_MEASUREMENTS_PATH)
display(raw_triangles_measurements)

,a,b,c,ha,hb,hc
0,91.10,81.30,74.30,63.50,70.90,77.35
1,91.20,81.35,74.25,63.00,71.00,77.50
2,90.90,81.15,74.20,63.25,70.90,77.60
3,91.10,81.15,74.30,63.50,71.90,77.30
4,91.20,81.30,74.30,63.40,71.40,77.60
5,90.75,80.90,74.10,63.70,70.70,77.50
6,90.90,80.00,74.00,63.00,70.65,77.30
7,91.30,81.25,74.35,63.30,70.90,77.50
8,91.00,81.20,74.15,63.15,70.75,77.30
9,91.00,81.20,74.20,63.05,70.90,77.65


## Odchylenia standardowe pomiarów

In [5]:
STANDARD_DEVIATION_SIDES_HEADERS = [
    "$$s_a \space [mm]$$",
    "$$s_b \space [mm]$$",
    "$$s_c \space [mm]$$",
    "$$s_{ha} \space [mm]$$",
    "$$s_{hb} \space [mm]$$",
    "$$s_{hc} \space [mm]$$",
]
standard_uncertainty = compute_standard_undertainty(raw_triangles_measurements)
display_table_as_markdown_with_headers(
    standard_uncertainty, STANDARD_DEVIATION_SIDES_HEADERS
)

|    |   $$s_a \space [mm]$$ |   $$s_b \space [mm]$$ |   $$s_c \space [mm]$$ |   $$s_{ha} \space [mm]$$ |   $$s_{hb} \space [mm]$$ |   $$s_{hc} \space [mm]$$ |
|---:|----------------------:|----------------------:|----------------------:|-------------------------:|-------------------------:|-------------------------:|
|  0 |              0.162512 |              0.372836 |              0.106969 |                 0.246774 |                 0.371458 |                  0.15494 |

## Średnie arytmetyczne pomiarów boków

In [6]:
means = compute_column_means(raw_triangles_measurements)
display_table_as_markdown_with_headers(means, LATEX_ARITHMETIC_MEANS_HEADERS)

|    |   $$\bar{a} \space [mm]$$ |   $$\bar{b} \space [mm]$$ |   $$\bar{c} \space [mm]$$ |   $$\bar{h}_a \space [mm]$$ |   $$\bar{h}_b \space [mm]$$ |   $$\bar{h}_c \space [mm]$$ |
|---:|--------------------------:|--------------------------:|--------------------------:|----------------------------:|----------------------------:|----------------------------:|
|  0 |                   91.0346 |                   81.1231 |                   74.2038 |                     63.2385 |                     70.9115 |                     77.4231 |

## Wskazanie pomiarów z błędami grubymi

Wyniki zawierające pomiary, których różnice od średniej przekraczają 3 odchylenia standardowe

In [7]:
display(mark_outliers(raw_triangles_measurements))

,|a - |,|b - |,|c - |,|ha - |,|hb - |,|hc - |
0,0.065385,0.176923,0.096154,0.261538,0.011538,0.073077
1,0.165385,0.226923,0.046154,0.238462,0.088462,0.076923
2,0.134615,0.026923,0.003846,0.011538,0.011538,0.176923
3,0.065385,0.026923,0.096154,0.261538,0.988462,0.123077
4,0.165385,0.176923,0.096154,0.161538,0.488462,0.176923
5,0.284615,0.223077,0.103846,0.461538,0.211538,0.076923
6,0.134615,1.123077,0.203846,0.238462,0.261538,0.123077
7,0.265385,0.126923,0.146154,0.061538,0.011538,0.076923
8,0.034615,0.076923,0.053846,0.088462,0.161538,0.123077
9,0.034615,0.076923,0.003846,0.188462,0.011538,0.226923


## Pomiary z wyeliminowanymi błędami grubymi

In [8]:
sanitized_triangles_measurements = remove_outliers(raw_triangles_measurements)
display_table_as_markdown_with_headers(
    sanitized_triangles_measurements, LATEX_MEASUREMENT_HEADERS
)

|    |   $$a \space [mm]$$ |   $$b \space [mm]$$ |   $$c \space [mm]$$ |   $$h_a \space [mm]$$ |   $$h_b \space [mm]$$ |   $$h_c \space [mm]$$ |
|---:|--------------------:|--------------------:|--------------------:|----------------------:|----------------------:|----------------------:|
|  0 |               91.1  |               81.3  |               74.3  |                 63.5  |                 70.9  |                 77.35 |
|  1 |               91.2  |               81.35 |               74.25 |                 63    |                 71    |                 77.5  |
|  2 |               90.9  |               81.15 |               74.2  |                 63.25 |                 70.9  |                 77.6  |
|  3 |               91.1  |               81.15 |               74.3  |                 63.5  |                 71.9  |                 77.3  |
|  4 |               91.2  |               81.3  |               74.3  |                 63.4  |                 71.4  |                 77.6  |
|  5 |               90.75 |               80.9  |               74.1  |                 63.7  |                 70.7  |                 77.5  |
|  7 |               91.3  |               81.25 |               74.35 |                 63.3  |                 70.9  |                 77.5  |
|  8 |               91    |               81.2  |               74.15 |                 63.15 |                 70.75 |                 77.3  |
|  9 |               91    |               81.2  |               74.2  |                 63.05 |                 70.9  |                 77.65 |
| 10 |               91.1  |               81.25 |               74.3  |                 63.25 |                 70.75 |                 77.5  |
| 11 |               91.1  |               81.55 |               74.1  |                 63.2  |                 70.5  |                 77.2  |
| 12 |               90.8  |               81    |               74.1  |                 62.8  |                 70.6  |                 77.2  |

## Niepewności standardowe całkowite

In [9]:
standard_uncertainty = compute_standard_measurement_uncertainty(
    sanitized_triangles_measurements, 0.05
)
display_table_as_markdown_with_headers(standard_uncertainty, LATEX_UNCERTAINTY_HEADERS)

|    |   $$u(\bar{a})$$ |   $$u(\bar{b})$$ |   $$u(\bar{c})$$ |   $$u( \bar{h_a})$$ |   $$u(\bar{h}_b)$$ |   $$u(\bar{h}_c)$$ |
|---:|-----------------:|-----------------:|-----------------:|--------------------:|-------------------:|-------------------:|
|  0 |         0.166913 |           0.1681 |         0.096039 |            0.248328 |           0.380291 |           0.159782 |

## Niepewności standardowe rozszerzone

In [10]:
extended_standard_uncertainties = compute_extended_standard_uncertainty(
    sanitized_triangles_measurements
)
display_table_as_markdown_with_headers(
    extended_standard_uncertainties, LATEX_STANDARD_UNCERTAINTY_HEADERS
)

|    |   $$u(a)$$ |   $$u(b)$$ |   $$u(c)$$ |   $$u(h_a)$$ |   $$u(h_b)$$ |   $$u(h_c)$$ |
|---:|-----------:|-----------:|-----------:|-------------:|-------------:|-------------:|
|  0 |   0.179815 |   0.181134 |   0.100188 |     0.269776 |     0.414756 |     0.171891 |

## Przedziały ufności

In [11]:
trust_intervals = pd.DataFrame()
for column in means.columns:
    trust_intervals[column] = [
        r"${:.2f} \pm {:.2f}$".format(val1, val2)
        for val1, val2 in zip(means[column], extended_standard_uncertainties[column])
    ]

display_table_as_markdown_with_headers(trust_intervals, LATEX_TRUST_INTERVALS_HEADERS)

|    | $$a \space [mm]$$   | $$b \space [mm]$$   | $$c \space [mm]$$   | $$h_a \space [mm]$$   | $$h_b \space [mm]$$   | $$h_c \space [mm]$$   |
|---:|:--------------------|:--------------------|:--------------------|:----------------------|:----------------------|:----------------------|
|  0 | $91.03 \pm 0.18$    | $81.12 \pm 0.18$    | $74.20 \pm 0.10$    | $63.24 \pm 0.27$      | $70.91 \pm 0.41$      | $77.42 \pm 0.17$      |

# Analiza pól trójkątów

In [12]:
raw_sufraces = compute_triangle_surface_all_methods(sanitized_triangles_measurements)
display_table_as_markdown_with_headers(raw_sufraces, LATEX_AREA_HEADERS)

|    |   $$S_{heron} \space [mm^2]$$ |   $$S_{ah} \space [mm^2]$$ |   $$S_{bh} \space [mm^2]$$ |   $$S_{ch} \space [mm^2]$$ |
|---:|------------------------------:|---------------------------:|---------------------------:|---------------------------:|
|  0 |                       2864.42 |                    2892.42 |                    2882.09 |                    2873.55 |
|  1 |                       2865.7  |                    2872.8  |                    2887.92 |                    2877.19 |
|  2 |                       2854.5  |                    2874.71 |                    2876.77 |                    2878.96 |
|  3 |                       2860.58 |                    2892.42 |                    2917.34 |                    2871.69 |
|  4 |                       2865.94 |                    2891.04 |                    2902.41 |                    2882.84 |
|  5 |                       2842.81 |                    2890.39 |                    2859.82 |                    2871.38 |
|  7 |                       2867.7  |                    2889.64 |                    2880.31 |                    2881.06 |
|  8 |                       2855.78 |                    2873.32 |                    2872.45 |                    2865.9  |
|  9 |                       2857.3  |                    2868.78 |                    2878.54 |                    2880.82 |
| 10 |                       2863.14 |                    2881.04 |                    2874.22 |                    2879.12 |
| 11 |                       2864.69 |                    2878.76 |                    2874.64 |                    2860.26 |
| 12 |                       2846.12 |                    2851.12 |                    2859.3  |                    2860.26 |

## Średnie wartości pól

In [13]:
surface_areas_means = compute_column_means(raw_sufraces)
display_table_as_markdown_with_headers(surface_areas_means, LATEX_AREA_HEADERS)

|    |   $$S_{heron} \space [mm^2]$$ |   $$S_{ah} \space [mm^2]$$ |   $$S_{bh} \space [mm^2]$$ |   $$S_{ch} \space [mm^2]$$ |
|---:|------------------------------:|---------------------------:|---------------------------:|---------------------------:|
|  0 |                       2859.06 |                     2879.7 |                    2880.48 |                    2873.59 |

## Średnie wartości na podstawie śrenich z pomiarów

In [14]:
# rename columns in means to be a,b,c, ha, hb, hc in mean
means_tmp = means.copy()
means_tmp.columns = ["a", "b", "c", "ha", "hb", "hc"]
surfrace_areas_from_side_means = compute_triangle_surface_all_methods(means_tmp)
display_table_as_markdown_with_headers(
    surfrace_areas_from_side_means, LATEX_AREA_HEADERS
)

|    |   $$S_{heron} \space [mm^2]$$ |   $$S_{ah} \space [mm^2]$$ |   $$S_{bh} \space [mm^2]$$ |   $$S_{ch} \space [mm^2]$$ |
|---:|------------------------------:|---------------------------:|---------------------------:|---------------------------:|
|  0 |                       2855.98 |                    2878.44 |                    2876.28 |                    2872.55 |

## Odchylenia standardowe pól

In [15]:
surface_standard_deviations = pd.DataFrame()
surface_standard_deviations["S_heron"] = np.sqrt(
    standard_uncertainty["a"] ** 2
    + standard_uncertainty["b"] ** 2
    + standard_uncertainty["c"] ** 2
)
surface_standard_deviations["S_ah"] = np.sqrt(
    standard_uncertainty["a"] ** 2 + standard_uncertainty["ha"] ** 2
)
surface_standard_deviations["S_bh"] = np.sqrt(
    standard_uncertainty["b"] ** 2 + standard_uncertainty["hb"] ** 2
)
surface_standard_deviations["S_ch"] = np.sqrt(
    standard_uncertainty["c"] ** 2 + standard_uncertainty["hc"] ** 2
)


display_table_as_markdown_with_headers(
    surface_standard_deviations, LATEX_SURFACE_STANDARD_DEVIATIONS_HEADERS
)

|    |   $$u(S_{heron}) \space [mm^2]$$ |   $$u(S_{ah}) \space [mm^2]$$ |   $$u(S_{bh}) \space [mm^2]$$ |   $$u(S_{ch}) \space [mm^2]$$ |
|---:|---------------------------------:|------------------------------:|------------------------------:|------------------------------:|
|  0 |                         0.255619 |                       0.29921 |                      0.415787 |                      0.186424 |

## Niepewności złozone

In [16]:
sem_uncertainties = raw_sufraces.sem()
sem_uncertainties = sem_uncertainties.multiply(3.7889823828484244)
sem_uncertainties = sem_uncertainties.to_frame().T

display_table_as_markdown_with_headers(
    sem_uncertainties, LATEX_EXTENDED_UNCERTAINTIES_HEADERS
)

|    |   $$S_{\bar{y}{Heron}}$$ |   $$S_{\bar{y}{ah}}$$ |   $$S_{\bar{y}{bh}}$$ |   $$S_{\bar{y}{ch}}$$ |
|---:|-------------------------:|----------------------:|----------------------:|----------------------:|
|  0 |                  8.79794 |               13.6675 |               17.8476 |               8.65698 |

## Przedziały ufności

In [17]:
trust_intervals = pd.DataFrame()
for column in surface_areas_means.columns:
    trust_intervals[column] = [
        r"${:.2f} \pm {:.2f}$".format(val1, val2)
        for val1, val2 in zip(surface_areas_means[column], sem_uncertainties[column])
    ]

LATEX_TRUST_INTERVALS_HEADERS = [
    r"$$S_{heron} \space [mm^2]$$",
    r"$$S_{ah} \space [mm^2]$$",
    r"$$S_{bh} \space [mm^2]$$",
    r"$$S_{ch} \space [mm^2]$$",
]
display_table_as_markdown_with_headers(trust_intervals, LATEX_TRUST_INTERVALS_HEADERS)

|    | $$S_{heron} \space [mm^2]$$   | $$S_{ah} \space [mm^2]$$   | $$S_{bh} \space [mm^2]$$   | $$S_{ch} \space [mm^2]$$   |
|---:|:------------------------------|:---------------------------|:---------------------------|:---------------------------|
|  0 | $2859.06 \pm 8.80$            | $2879.70 \pm 13.67$        | $2880.48 \pm 17.85$        | $2873.59 \pm 8.66$         |